Como en ninguno de mis datasets veo q me puedo aprobechar del uso de una api externa voy a extaer tweets relacionados a una empresa de internet local (Telecentro) lo cual seria el caso mas cercano de un uso de api a mi dataset de churn que proviene de una empresa de el mismo rubro (pero no decia en ningun lado el nombre de la empresa)
>
En este caso voy a empezar orientando la extracción a un sentiment analysis para medir la satisfacción de los usuarios por zonas y como hablan de la empresa.
>
Tambien se podria trackear a la competencia para ver que promociones e impacto en la comunidad tienen.

In [1]:
!pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

In [3]:
import tweepy

KEY = 'x8BfNYzKdXPlR3wAQW8ouS46P'
SECRET = 'Zn9GNVeTJ6KZ6S1Zm8ys28BEoJNeZzpTjeYxVFJWIVKQOcQdKi'

auth = tweepy.AppAuthHandler(KEY, SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)

Voy a ver que me da cada uno

In [4]:
#excluyo entidades pq molestaban mas que ayudaban
tweets = api.search(q='telecentro',lang='es',count=10,include_entities=False)

In [5]:
tweets[4]._json

{'created_at': 'Thu Oct 13 17:37:51 +0000 2022',
 'id': 1580613804207280129,
 'id_str': '1580613804207280129',
 'text': 'Telecentro Altavista los Tanques te informa nuestros horarias de atención al público.\n¡Ven a nuestras instalaciones… https://t.co/fSItKHIpcK',
 'truncated': True,
 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1518996528442101760,
  'id_str': '1518996528442101760',
  'name': 'telecentro los tanques',
  'screen_name': 'Telecen_Tanques',
  'location': '',
  'description': 'Telecentro Altavista los Tanques, pone la Tegnología al servicio de la comunidad.',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 0,
  'friends_count': 0,
  'l

Filtro el json ya que no quiero todo

In [6]:
#preparo las columnas que quiero que tenga el dataframe final
dict_df_final = {
    'tweet_id':[],
    'text':[],
    'fecha_creacion':[],
    'n_veces_retweeteado':[],
    'n_veces_favoriteado':[],
    'locacion_tweet':[],
    'in_reply_to_screen_name':[],
    'user':[],
    'user_id_creacion':[],
    'locacion_usuario':[],
    'n_seguidores_usuario':[],
    'n_seguidos_por_usuario':[],
    'fecha_usuario_creado':[],
    'n_favoritos_usuario':[],
    'n_tweets_usuario':[],
    }

¿Por qué elegí esas columnas?

* tweet_id/user_id_creacion/usuario : identificadores del tweet y de quien lo posteo

* text: lo q se va a analizar con el sentiment analisis
* fecha_creación: cuando ocurrió el tweet
* n_veces_retweeteado/favoriteado: el tamaño impacto en la comunidad, a cuantas personas llego (a mas retweets = a mas gente le llego)
* in_reply_to_screen_name: nombre de usuario al que le responde para filtrar los mensajes al servicio tecnico por tweeter q ofrece la empresa.
* locacion_tweet/usuario: para intentar saber en q zona esta ocurriendo el problema, la locación del tweet seria la zona principal y en caso de ser nula la locacion usuario puede servir como segunda opcion.
* n_seguidores_usuario , n_seguidos_por_usuario, n_tweets_usuario, n_favoritos_usuario , fecha_usuario_creado: pueden ser usados como metricas para distinguir si un usuario es un potencial bot o real.

In [7]:
def parse_json(data):
  dict_df_final['tweet_id'].append(data['id'])
  dict_df_final['text'].append(data['text'])
  dict_df_final['fecha_creacion'].append(data['created_at'])
  dict_df_final['n_veces_retweeteado'].append(data['retweet_count'])
  dict_df_final['n_veces_favoriteado'].append(data['favorite_count'])
  dict_df_final['locacion_tweet'].append(data['place'])
  dict_df_final['in_reply_to_screen_name'].append(data['in_reply_to_screen_name'])
  dict_df_final['user_id_creacion'].append(data['user']['id'])
  dict_df_final['user'].append(data['user']['screen_name'])
  dict_df_final['locacion_usuario'].append(data['user']['location'])
  dict_df_final['n_seguidores_usuario'].append(data['user']['followers_count'])
  dict_df_final['n_seguidos_por_usuario'].append(data['user']['friends_count'])
  dict_df_final['fecha_usuario_creado'].append(data['user']['created_at'])
  dict_df_final['n_favoritos_usuario'].append(data['user']['favourites_count'])
  dict_df_final['n_tweets_usuario'].append(data['user']['statuses_count'])

In [8]:
#veo como se comporta a menor escala
for tweet in tweets:
  parse_json(tweet._json)

pd.DataFrame(dict_df_final)

,tweet_id,text,fecha_creacion,n_veces_retweeteado,n_veces_favoriteado,locacion_tweet,in_reply_to_screen_name,user,user_id_creacion,locacion_usuario,n_seguidores_usuario,n_seguidos_por_usuario,fecha_usuario_creado,n_favoritos_usuario,n_tweets_usuario
0,1580619310061408257,Telecentro directamente SACÓ la opción dar de ...,Thu Oct 13 17:59:44 +0000 2022,0,0,None,None,issues_ps,78153319,Buenos Aires,929,3675,Mon Sep 28 23:25:30 +0000 2009,117293,21453
1,1580617985487962113,@LastpickGG De Buenos Aires juego a 27-30 ms c...,Thu Oct 13 17:54:28 +0000 2022,0,0,None,LastpickGG,agustin552uno,501035513,,81,102,Thu Feb 23 18:35:16 +0000 2012,7965,744
2,1580615784790839301,Invitados todas y todos para que iniciemos pro...,Thu Oct 13 17:45:44 +0000 2022,0,0,None,None,losmangostelec,832671741398822914,Calle 57C # 24-17,3,42,Fri Feb 17 19:23:08 +0000 2017,14,71
3,1580614114107195392,RT @MarielMer1: Me dan una manito en el tuit d...,Thu Oct 13 17:39:05 +0000 2022,29,0,None,None,nelunelu57,2955119907,,1185,1279,Thu Jan 01 17:30:23 +0000 2015,60109,28510
4,1580613804207280129,Telecentro Altavista los Tanques te informa nu...,Thu Oct 13 17:37:51 +0000 2022,0,0,None,None,Telecen_Tanques,1518996528442101760,,0,0,Tue Apr 26 16:53:55 +0000 2022,4,35
5,1580612568158457856,"@NLBZ26 @NLBZ26, ¡Síguenos! en @Telecentroayud...",Thu Oct 13 17:32:57 +0000 2022,0,0,None,NLBZ26,TeleCentroAyuda,557678032,"Buenos Aires, Argentina",72712,64746,Thu Apr 19 12:33:40 +0000 2012,1950,785731
6,1580612282169847815,@PersonalAr me voy a Telecentro peor que usted...,Thu Oct 13 17:31:49 +0000 2022,0,0,None,PersonalAr,ines_andrea_,1534287795430252544,,170,307,Tue Jun 07 21:35:41 +0000 2022,696,3688
7,1580611893093224448,Telecentro Altavista los Tanques te saluda y t...,Thu Oct 13 17:30:16 +0000 2022,0,0,None,None,Telecen_Tanques,1518996528442101760,,0,0,Tue Apr 26 16:53:55 +0000 2022,4,35
8,1580611720896450560,@TeleCentroAyuda es increíble que con el celul...,Thu Oct 13 17:29:35 +0000 2022,0,0,None,TeleCentroAyuda,Geo_Rod,139271621,"Barquisimeto,Venezuela",220,412,Sun May 02 03:05:06 +0000 2010,377,1100


In [9]:
#reseto el dict y hago el querry grande
dict_df_final = {
    'tweet_id':[],
    'text':[],
    'fecha_creacion':[],
    'n_veces_retweeteado':[],
    'n_veces_favoriteado':[],
    'locacion_tweet':[],
    'in_reply_to_screen_name':[],
    'user':[],
    'user_id_creacion':[],
    'locacion_usuario':[],
    'n_seguidores_usuario':[],
    'n_seguidos_por_usuario':[],
    'fecha_usuario_creado':[],
    'n_favoritos_usuario':[],
    'n_tweets_usuario':[],
    }
    
tweets = api.search(q='telecentro',lang='es',count=500,include_entities=False)

for tweet in tweets:
  parse_json(tweet._json)

df = pd.DataFrame(dict_df_final)

In [10]:
df

,tweet_id,text,fecha_creacion,n_veces_retweeteado,n_veces_favoriteado,locacion_tweet,in_reply_to_screen_name,user,user_id_creacion,locacion_usuario,n_seguidores_usuario,n_seguidos_por_usuario,fecha_usuario_creado,n_favoritos_usuario,n_tweets_usuario
0,1580619310061408257,Telecentro directamente SACÓ la opción dar de ...,Thu Oct 13 17:59:44 +0000 2022,0,0,None,None,issues_ps,78153319,Buenos Aires,929,3675,Mon Sep 28 23:25:30 +0000 2009,117293,21453
1,1580617985487962113,@LastpickGG De Buenos Aires juego a 27-30 ms c...,Thu Oct 13 17:54:28 +0000 2022,0,0,None,LastpickGG,agustin552uno,501035513,,81,102,Thu Feb 23 18:35:16 +0000 2012,7965,744
2,1580615784790839301,Invitados todas y todos para que iniciemos pro...,Thu Oct 13 17:45:44 +0000 2022,0,0,None,None,losmangostelec,832671741398822914,Calle 57C # 24-17,3,42,Fri Feb 17 19:23:08 +0000 2017,14,71
3,1580614114107195392,RT @MarielMer1: Me dan una manito en el tuit d...,Thu Oct 13 17:39:05 +0000 2022,29,0,None,None,nelunelu57,2955119907,,1185,1279,Thu Jan 01 17:30:23 +0000 2015,60109,28510
4,1580613804207280129,Telecentro Altavista los Tanques te informa nu...,Thu Oct 13 17:37:51 +0000 2022,0,0,None,None,Telecen_Tanques,1518996528442101760,,0,0,Tue Apr 26 16:53:55 +0000 2022,4,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1580488961147179008,@Telecentro_Ayu @TeleCentroAr @TeleCentroAyuda...,Thu Oct 13 09:21:47 +0000 2022,0,0,None,Telecentro_Ayu,ElyyyEliana,1570543988859441158,,1,4,Thu Sep 15 22:44:30 +0000 2022,0,3
96,1580485287318151168,RT @MarielMer1: Me dan una manito en el tuit d...,Thu Oct 13 09:07:11 +0000 2022,29,0,None,None,gladysnfigueroa,1430656944784744455,,876,901,Wed Aug 25 22:23:50 +0000 2021,12429,46628
97,1580482123462758400,RT @TheCrowAR: dale @TeleCentroAyuda media pil...,Thu Oct 13 08:54:36 +0000 2022,5,0,None,None,javitunas,251929464,"Buenos Aires, Argentina",755,2346,Mon Feb 14 03:47:45 +0000 2011,48514,20555
98,1580473975804612609,RT @TheCrowAR: dale @TeleCentroAyuda media pil...,Thu Oct 13 08:22:14 +0000 2022,5,0,None,None,SalchiDachshund,1115502084416266240,"Buenos Aires, Argentina",2605,5001,Tue Apr 09 06:29:56 +0000 2019,90004,80358


bueno, la api solo me deja los ultimos 100 tweets sin pagar 🙂